In [102]:
from dotenv import load_dotenv
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import SystemMessage, trim_messages

import os

In [103]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="pr-juicy-equality-90"

os.environ["NVIDIA_API_KEY"] = "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"

In [104]:
# initialise NVIDIA model for chat
model = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")

In [105]:
# send the first message
model.invoke([HumanMessage(content="Hi! I am Deepak")])

AIMessage(content="Namaste Deepak! It's nice to meet you. How are you doing today? Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Namaste Deepak! It's nice to meet you. How are you doing today? Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 17, 'total_tokens': 50, 'completion_tokens': 33}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-a8f60ef1-fb20-4893-990a-b59e36244e49-0', usage_metadata={'input_tokens': 17, 'output_tokens': 33, 'total_tokens': 50}, role='assistant')

In [106]:
# send follow up message to see if the model remembers - it wont
model.invoke([HumanMessage(content="What was my name?")])

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have personal interactions or memories, and I don't retain information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to tell me your name, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "I don't have any information about your name. I'm a large language model, I don't have personal interactions or memories, and I don't retain information about individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. If you'd like to tell me your name, I'd be happy to chat with you!", 'token_usage': {'prompt_tokens': 16, 'total_tokens': 94, 'completion_tokens': 78}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-6540a1a0-be4d-4dd0-aa11-b946cd82f3d2-0', usa

In [107]:
# send full message history to get the context
model.invoke(
    [
        HumanMessage(content="Hi! I am Deepak"),
        AIMessage(content="Hi Deepak! Nice to meet you! Is there something I can help you with?"),
        HumanMessage(content="What was my name?")
    ]

)

AIMessage(content='Your name is Deepak!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Your name is Deepak!', 'token_usage': {'prompt_tokens': 50, 'total_tokens': 56, 'completion_tokens': 6}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-fdbd93f5-2c85-479e-9e6c-82d14dd9cf86-0', usage_metadata={'input_tokens': 50, 'output_tokens': 6, 'total_tokens': 56}, role='assistant')

## Prompt template

In [108]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful translator. Answer all questions in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

## Message Persistence

In [109]:
trimmer = trim_messages(
    max_tokens = 100,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

In [127]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    print(f"TRIM: ========={trimmed_messages[-1]}=========")
    response = chain.invoke(
        {"messages":trimmed_messages, "language":state["language"]}
    )
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

SyntaxError: f-string: unmatched '[' (1335967762.py, line 10)

In [124]:
# This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.
config = {"configurable": {"thread_id":"abc123"}}

In [122]:
query = "Hi! I am Myra"
language = "English"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language": language}, config)

output["messages"]#[-1].pretty_print()

TRIM: =========[HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='d573e0d4-02b3-466a-ab7e-3d8da8b97681'), HumanMessage(content='Hi! I am Myra', additional_kwargs={}, response_metadata={}, id='977e4a0d-da1b-46db-804c-5aaf51b25d85')]=========


[HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='d573e0d4-02b3-466a-ab7e-3d8da8b97681'),
 HumanMessage(content='Hi! I am Myra', additional_kwargs={}, response_metadata={}, id='977e4a0d-da1b-46db-804c-5aaf51b25d85'),
 AIMessage(content='Nice to meet you, Myra! How can I assist you with translation today?', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Nice to meet you, Myra! How can I assist you with translation today?', 'token_usage': {'prompt_tokens': 44, 'total_tokens': 61, 'completion_tokens': 17}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-9ae33da5-620b-49e6-a656-c1efb66e5510-0', usage_metadata={'input_tokens': 44, 'output_tokens': 17, 'total_tokens': 61}, role='assistant')]

In [125]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

output["messages"]#[-1].pretty_print()

TRIM: =========[HumanMessage(content='Hi! I am Myra', additional_kwargs={}, response_metadata={}, id='aca961b1-110d-4c42-a77b-46d89edcbbf5'), AIMessage(content="Hello Myra! Nice to meet you! I'm happy to chat with you and help with any questions you might have. How's your day going so far? Do you have something on your mind that you'd like to talk about or ask about? I'm all ears!", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hello Myra! Nice to meet you! I'm happy to chat with you and help with any questions you might have. How's your day going so far? Do you have something on your mind that you'd like to talk about or ask about? I'm all ears!", 'token_usage': {'prompt_tokens': 117, 'total_tokens': 173, 'completion_tokens': 56}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-a57b7c31-09b6-445c-8212-8a876508a446-0', usage_metadata={'input_tokens': 117, 'output_tokens': 56, 'total_tokens': 173}, role='assistant'), HumanMessa

[HumanMessage(content='Hi! I am Myra', additional_kwargs={}, response_metadata={}, id='e379d1cb-bd48-42e9-afb5-ebd8935799a1'),
 AIMessage(content='Hello Myra! Nice to meet you. How can I help you today? Do you need some translation help or just want to chat?', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Hello Myra! Nice to meet you. How can I help you today? Do you need some translation help or just want to chat?', 'token_usage': {'prompt_tokens': 34, 'total_tokens': 62, 'completion_tokens': 28}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-8c38c672-5f9f-4b49-bd21-5293f5c40e64-0', usage_metadata={'input_tokens': 34, 'output_tokens': 28, 'total_tokens': 62}, role='assistant'),
 HumanMessage(content="My dad's name is Deepak", additional_kwargs={}, response_metadata={}, id='fcdbac05-2e98-4cad-a32e-041e829f015a'),
 AIMessage(content='That\'s nice! Deepak is a popular Indian name. It means "lamp" or "light" in Sanskrit. Is the